In [11]:
#| default_exp BaltNet
#| default_cls_lvl 3

# BaltNet

Model architecture used for predicting river runoff in the Baltic Sea

In [12]:
#| hide
from nbdev import show_doc

In [13]:
#| export

import torch
import torch.nn as nn
import lightning as L
import pytorch_lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import numpy as np

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import torch.nn.functional as F
import torchmetrics
from lightning.pytorch.loggers import CSVLogger

import xarray as xr
from glob import glob
from tqdm import tqdm

from BalticRiverPrediction.convLSTM import ConvLSTM
from BalticRiverPrediction.sharedUtilities import read_netcdfs, preprocess, plot_loss_and_acc

In [1]:
#| export
class BaltNet(nn.Module):
    def __init__(self, modelPar):
        super(BaltNet, self).__init__()

        # Initialize all attributes
        for k, v in modelPar.items():
            setattr(self, k, v)

        self.encoder = ConvLSTM(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            kernel_size=self.kernel_size,
            num_layers=self.num_layers,
            batch_first=self.batch_first,
            bias=self.bias,
            return_all_layers=False
        )

        self.decoder = ConvLSTM(
            input_dim=self.hidden_dim,
            hidden_dim=self.hidden_dim,
            kernel_size=self.kernel_size,
            num_layers=1,
            batch_first=self.batch_first,
            bias=self.bias,
            return_all_layers=False
        )

        self.linear_dim = self.dimensions[0] * self.dimensions[1] * self.hidden_dim 

        # Single fully connected network for all rivers
         
        self.river_predictors = nn.Sequential(
            nn.Linear(self.linear_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 97)
        )

        # Creating separate attention weights for each river
        self.attention_weights = nn.Parameter(torch.randn(self.hidden_dim, 1, 1), requires_grad=True)  # 97 rivers

    def spatial_attention(self, x):
        """Spatial attention mechanism."""
        B, T, C, H, W = x.size()

        x = x.view(B * T, C, H, W)
        
        # Apply attention weights for all rivers
        self.attention_map = torch.sigmoid(F.conv2d(x, self.attention_weights.unsqueeze(0), bias=None, stride=1, padding=0))
        
        # Weighted sum
        output = x * self.attention_map  # B*T, C, H, W
        output = output.view(B, T, C, H, W)  # B, T, C

        return output

    def forward(self, x):
        B, _, _, _, _ = x.size()

        # Pass through encoder
        encoder_outputs, encoder_hidden = self.encoder(x)

        # Use the entire encoder output as input to the decoder
        decoder_input = encoder_outputs[0][:,-1,:,:,:].unsqueeze(1)

        # Pass through decoder using the final hidden state of the encoder
        decoder_outputs, _ = self.decoder(decoder_input, encoder_hidden)

        # Apply spatial attention
        decoder_with_spatial_attention = self.spatial_attention(decoder_outputs[0])  # B, T, C, H, W
            
        # Flatten the temporal sequence
        decoder_with_spatial_attention_flattened = decoder_with_spatial_attention.view(B, -1)  #
            
        # Pass through its own predictor
        output = self.river_predictors(decoder_with_spatial_attention_flattened)  # B, -1

        return output

NameError: name 'nn' is not defined

In [15]:
show_doc(BaltNet)

---

### BaltNet

>      BaltNet (modelPar)

A custom neural network architecture designed for sequence-to-sequence learning using convolutional LSTMs.

Attributes:
- encoder: A convolutional LSTM serving as the encoder.
- decoder: A convolutional LSTM serving as the decoder.
- river_predictors: A feed-forward network for predicting river values.
- attention_weights: Weights for the spatial attention mechanism.

Parameters:
- modelPar (dict): Dictionary containing the model parameters.

In [16]:
show_doc(BaltNet.spatial_attention)

---

### BaltNet.spatial_attention

>      BaltNet.spatial_attention (x)

Applies the spatial attention mechanism to the input tensor.

Parameters:
- x (torch.Tensor): Input tensor of shape `(batch_size, sequence_length, channels, height, width)`.

Returns:
- output (torch.Tensor): Input tensor after applying the spatial attention mechanism.

In [17]:
show_doc(BaltNet.forward)

---

### BaltNet.forward

>      BaltNet.forward (x)

Forward pass of the BaltNet.

Parameters:
- x (torch.Tensor): Input tensor of shape `(batch_size, sequence_length, channels, height, width)`.

Returns:
- output (torch.Tensor): Predicted values for the rivers.

In [20]:
#| export
class LightningModel(L.LightningModule):
    """
    A PyTorch Lightning model for training and evaluation.
    
    Attributes:
        model (nn.Module): The neural network model.
        learning_rate (float): Learning rate for the optimizer.
        cosine_t_max (int): Maximum number of iterations for the cosine annealing scheduler.
        train_mse (torchmetrics.MeanSquaredError): Metric for training mean squared error.
        val_mse (torchmetrics.MeanSquaredError): Metric for validation mean squared error.
        test_mse (torchmetrics.MeanSquaredError): Metric for testing mean squared error.
    """
    
    def __init__(self, model, learning_rate, cosine_t_max):
        """
        Initializes the LightningModel.

        Args:
            model (nn.Module): The neural network model.
            learning_rate (float): Learning rate for the optimizer.
            cosine_t_max (int): Maximum number of iterations for the cosine annealing scheduler.
        """
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model
        self.cosine_t_max = cosine_t_max

        # Save hyperparameters except the model
        self.save_hyperparameters(ignore=["model"])

        # Define metrics
        self.train_mse = torchmetrics.MeanSquaredError()
        self.val_mse = torchmetrics.MeanSquaredError()
        self.test_mse = torchmetrics.MeanSquaredError()

    def forward(self, x):
        """Defines the forward pass of the model."""
        return self.model(x)
    
    def _shared_step(self, batch, debug=False):
        """
        Shared step for training, validation, and testing.

        Args:
            batch (tuple): Input batch of data.
            debug (bool, optional): If True, prints the loss. Defaults to False.

        Returns:
            tuple: Computed loss, true labels, and predicted labels.
        """
        features, true_labels = batch
        logits = self.model(features)
        loss = F.mse_loss(logits, true_labels)
        if debug:
            print(loss)
        return loss, true_labels, logits
    
    def training_step(self, batch, batch_idx):
        """Training step."""
        loss, true_labels, predicted_labels = self._shared_step(batch)
        mse = self.train_mse(predicted_labels, true_labels)
        metrics = {"train_mse": mse, "train_loss": loss}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar=True, logger=True, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        """Validation step."""
        loss, true_labels, predicted_labels = self._shared_step(batch)
        mse = self.val_mse(predicted_labels, true_labels)
        self.log("val_loss", loss, sync_dist=True)
        self.log("val_mse", mse, prog_bar=True, sync_dist=True)
    
    def test_step(self, batch, _):
        """Test step."""
        loss, true_labels, predicted_labels = self._shared_step(batch)
        mse = self.test_mse(predicted_labels, true_labels)
        self.log("test_loss", loss, rank_zero_only=True)
        self.log("test_mse", mse, sync_dist=True)
        return loss
    
    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = 0):
        """Prediction step."""
        _, _, predicted_labels = self._shared_step(batch)
        return predicted_labels

    def configure_optimizers(self):
        """
        Configures the optimizer and learning rate scheduler.

        Returns:
            tuple: List of optimizers and list of learning rate schedulers.
        """
        opt = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        sch = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10, verbose=False)
        return {"optimizer": opt, "lr_scheduler": sch, "monitor": "val_mse"}

In [21]:
show_doc(LightningModel)

---

### LightningModel

>      LightningModel (model, learning_rate, cosine_t_max)

A custom PyTorch Lightning model designed for the encapsulation of training, validation, and testing logic of a neural network.

Attributes:
- model (nn.Module): The main neural network.
- learning_rate (float): Learning rate used for optimization.
- cosine_t_max (int): Maximum number of iterations for the cosine annealing scheduler.
- train_mse, val_mse, test_mse (torchmetrics.MeanSquaredError): Metrics for computing the mean squared error during training, validation, and testing respectively.

In [22]:
show_doc(LightningModel._shared_step)

---

### LightningModel._shared_step

>      LightningModel._shared_step (batch, debug=False)

A shared method for computation during training, validation, and testing.

Args:
    batch (tuple): Input data batch.
    debug (bool, optional): If set to True, the method will print the loss value. Defaults to False.

Returns:
    tuple: Computed loss, actual labels, and predicted labels.

In [25]:
#| export
class AtmosphericDataset(Dataset):
    def __init__(self, input_size, atmosphericData, runoff, transform=None):

        # Length of the sequence
        self.input_size = input_size

        # input data (x) 
        atmosphericDataStd = atmosphericData.std("time") # dimension will be channel, lat, lon
        atmosphericDataMean = atmosphericData.mean("time")
        self.atmosphericStats = (atmosphericDataMean, atmosphericDataStd)

        # output data - label (y)
        runoffData = runoff.transpose("time", "river")
        runoffDataMean = runoffData.mean("time")
        runoffDataSTD = runoffData.std("time")
        self.runoffDataStats = (runoffDataMean, runoffDataSTD)
        
        # normalize data
        X = ((atmosphericData - atmosphericDataMean)/atmosphericDataStd).compute()
        y = ((runoffData - runoffDataMean)/runoffDataSTD).compute()
        
        # an additional dimension for the channel is added
        # to end up with (time, channel, lat, lon)
        xStacked = X.to_array(dim='variable')
        xStacked = xStacked.transpose("time", "variable", "y", "x")

        assert xStacked.data.ndim == 4
        self.x = torch.tensor(xStacked.data, dtype=torch.float32)
        self.y = torch.tensor(y.data, dtype=torch.float32)

    def __getitem__(self, index):
        return self.x[index:index+(self.input_size)], self.y[index+int(self.input_size)]

    def __len__(self):
        return self.y.shape[0]-(self.input_size)

In [26]:
show_doc(AtmosphericDataset)

---

### AtmosphericDataset

>      AtmosphericDataset (input_size, atmosphericData, runoff, transform=None)

Dataset for atmospheric data and runoff prediction.

Attributes:
- input_size (int): Length of the input sequence.
- atmosphericStats (tuple): Mean and standard deviation of the atmospheric data along the time dimension.
- runoffDataStats (tuple): Mean and standard deviation of the runoff data along the time dimension.
- x (torch.Tensor): Normalized atmospheric data tensor with dimensions (time, channel, lat, lon).
- y (torch.Tensor): Normalized runoff data tensor with dimensions (time, river).

In [27]:
#| export

class AtmosphereDataModule(L.LightningDataModule):
    
    def __init__(self, atmosphericData, runoff, batch_size=64, num_workers=8, add_first_dim=True, input_size=30):
        super().__init__()

        self.data = atmosphericData
        self.runoff = runoff
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.add_first_dim = add_first_dim
        self.input_size = input_size
    
    def setup(self, stage:str):
        UserWarning("Loading atmospheric data ...")
        dataset = AtmosphericDataset(
            atmosphericData=self.data,
            runoff=self.runoff,
            input_size=self.input_size
            )
        n_samples = len(dataset)
        train_size = int(0.85 * n_samples)
        val_size = n_samples - train_size
        # self.train, self.val = train_test_split(dataset)
        self.train, self.val, = random_split(dataset, [train_size, val_size])

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train,
            batch_size=self.batch_size,
            shuffle=True, 
            drop_last=True, 
            num_workers=self.num_workers,
            pin_memory=False  # Speed up data transfer to GPU
        )
    
    def val_dataloader(self):
        return DataLoader(
            dataset=self.val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            drop_last=True,
            pin_memory=False  # Speed up data transfer to GPU
        )

In [28]:
show_doc(AtmosphereDataModule)

---

### AtmosphereDataModule

>      AtmosphereDataModule (atmosphericData, runoff, batch_size=64,
>                            num_workers=8, add_first_dim=True, input_size=30)

A PyTorch Lightning DataModule for preparing and loading atmospheric and runoff datasets.

Attributes:
    data (xr.DataArray): The atmospheric data.
    runoff (xr.DataArray): The runoff data corresponding to the atmospheric data.
    batch_size (int): The batch size for the DataLoader.
    num_workers (int): Number of subprocesses to use for data loading.
    add_first_dim (bool): Flag to determine if a first dimension needs to be added. (Not used in current implementation).
    input_size (int): The input sequence length for the `AtmosphericDataset`.
    train (Dataset): Training dataset.
    val (Dataset): Validation dataset.